In [ ]:
%load_ext autoreload
%autoreload 2
from skimage import data
import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread, imwrite
from scipy import ndimage
import helperfunctions as hf

### Load two adjacent tiles

In [ ]:
left = imread('images/upscaled/left_up_noise6.tif')
right = imread('images/upscaled/right_up_noise6.tif')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(left, cmap='gray')
ax2.imshow(right, cmap='gray')

In [ ]:
image_map = {
    'left': left,
    'right': right
}

image_positions = {
    'left': (0, 0),
    'right': (0, 400)
}

im_size = left.shape
output_shape = (im_size[0], im_size[1] + 400)

shifted_images = []
for image in ['left', 'right']:
    im_shifted = ndimage.affine_transform(
        input = image_map[image],
        matrix = np.eye(2),
        offset = -np.array([0,0]) - np.array(image_positions[image]),
        output_shape = output_shape
    )
    shifted_images.append(im_shifted)

output = np.max(np.stack(shifted_images, axis = 2), axis = 2)
plt.imshow(output)

In [ ]:


left_gradient = np.tile(np.linspace(1, 0, 112), (512, 1))
right_gradient = np.tile(np.linspace(0, 1, 112), (512, 1))

fused_dims_y = 512
fused_dims_x = 912
left = imread('images/upscaled/left_up_noise6.tif')
right = imread('images/upscaled/right_up_noise6.tif')

left_corr = left[:, 400:] * left_gradient
right_corr = right[:, :112] * right_gradient

left[:, 400:] = left_corr
right[:, :112] = right_corr

translation =  fused_dims_x - fused_dims_y
left_shifted = ndimage.affine_transform(
    input = left,
    matrix = np.eye(2),
    offset = -np.array([0, 0]),
    output_shape = (fused_dims_y, fused_dims_x)
)
right_shifted = ndimage.affine_transform(
    input = right,
    matrix = np.eye(2),
    offset = -np.array([0, translation]),
    output_shape = (fused_dims_y, fused_dims_x)
)
output = np.sum(np.stack([left_shifted, right_shifted], axis=2), axis=2)
boundary_start = translation
boundary_end = 512 + (512 - translation)
print(boundary_start, boundary_end) 
plt.imshow(output); plt.show()

In [ ]:
import numpy as np
left_gradient = np.tile(np.linspace(1, 0, 112), (512, 1))
right_gradient = np.tile(np.linspace(0, 1, 224), (512, 1))
plt.imshow(left_gradient); plt.show()
plt.imshow(right_gradient); plt.show()
left_gradient.shape